In [ ]:
import numpy as np

SBOX_DEC = np.array([ # Int-version of the AES SBox 
    [ 99, 124, 119, 123, 242, 107, 111, 197,  48,   1, 103,  43, 254, 215, 171, 118],
    [202, 130, 201, 125, 250,  89,  71, 240, 173, 212, 162, 175, 156, 164, 114, 192],
    [183, 253, 147,  38,  54,  63, 247, 204,  52, 165, 229, 241, 113, 216,  49,  21],
    [  4, 199,  35, 195,  24, 150,   5, 154,   7,  18, 128, 226, 235,  39, 178, 117],
    [  9, 131,  44,  26,  27, 110,  90, 160,  82,  59, 214, 179,  41, 227,  47, 132],
    [ 83, 209,   0, 237,  32, 252, 177,  91, 106, 203, 190,  57,  74,  76,  88, 207],
    [208, 239, 170, 251,  67,  77,  51, 133,  69, 249,   2, 127,  80,  60, 159, 168],
    [ 81, 163,  64, 143, 146, 157,  56, 245, 188, 182, 218,  33,  16, 255, 243, 210],
    [205,  12,  19, 236,  95, 151,  68,  23, 196, 167, 126,  61, 100,  93,  25, 115],
    [ 96, 129,  79, 220,  34,  42, 144, 136,  70, 238, 184,  20, 222,  94,  11, 219],
    [224,  50,  58,  10,  73,   6,  36,  92, 194, 211, 172,  98, 145, 149, 228, 121],
    [231, 200,  55, 109, 141, 213,  78, 169, 108,  86, 244, 234, 101, 122, 174,   8],
    [186, 120,  37,  46,  28, 166, 180, 198, 232, 221, 116,  31,  75, 189, 139, 138],
    [112,  62, 181, 102,  72,   3, 246,  14,  97,  53,  87, 185, 134, 193,  29, 158],
    [225, 248, 152,  17, 105, 217, 142, 148, 155,  30, 135, 233, 206,  85,  40, 223],
    [140, 161, 137,  13, 191, 230,  66, 104,  65, 153,  45,  15, 176,  84, 187,  22]
]) 


SUPPORTED_TARGETS = ['SBO', 'HW']


def compute_labels(plaintext, key, target='SBO'):
    
    """ Computation of the targets for each byte of the key
    Input: 
        - plaintext: numpy array of int relative to each byte of the hex plaintext
        - key: numpy array of int relative to each byte of the hex key
        - target: Target of the network

    Output: numpy array of int relative to the specified target for each byte of the key
        - 'SOB': numpy array containing the SBox-lookup outputs 
        - 'HW': numpy array containing the Hamming Weights of the SBox-lookup outputs
    """

    assert target in SUPPORTED_TARGETS, 'Given target is not supported!'

    sbox_out = compute_sbox_out(plaintext, key)

    if target == 'SBO':
        return sbox_out
    elif target == 'HW':
        return compute_hamming_weights(sbox_out)


def compute_sbox_out(plaintext, key):

    """ SBox-lookup outputs computation given plaintext and key
    Input:
        - plaintext: 128 bit (16 bytes) in hex format (single string without separators)
        - key: 128 bit (16 bytes) in hex format (single string without separators)

    Output: numpy array containing the SBox-lookup outputs relative to the given plaintext and key
    """

    sbox_in = add_round_key(plaintext, key) # AES AddRoundKey
    sbox_out = sub_bytes(sbox_in) # AES SubBytes

    return sbox_out
        
        
def add_round_key(plaintext, key):

    """ Implementation of AES AddRoundKey (plaintext XOR key)
    Input:
        - plaintext: numpy array of int relative to each byte of the hex plaintext
        - key: numpy array of int relative to each byte of the hex key

    Output: numpy array containing each byte of the XOR between plaintext and key (int values) 
    """

    return plaintext ^ key
    
    
def sub_bytes(sbox_in):

    """ Implementation of AES SubBytes (SBox-lookup)
    Input:
        - sbox_in: numpy array containing each byte of the XOR between plaintext and key (int values) 

    Output: Result of the SBox-lookup 
    """

    sbox_in_hex = format_hex(sbox_in) # Convert the SBox input to well-formatted hex (each byte independently)

    rows = [int(byte[0], 16) for byte in sbox_in_hex] # The first 4 bits (of each byte) are the row index 
    cols = [int(byte[1], 16) for byte in sbox_in_hex] # The remaining 4 bits (of each byte) are the col index 

    return SBOX_DEC[rows, cols]
        
        
def format_hex(int_bytes_array):

    """ Conversion of an array of int to an array of hex, where the eventual 0 in front is explicit
    Input:
        - int_bytes_array: numpy array of int (one int refers to a single byte of a hex string)

    Output: list of well-formatted hex values relative to the input int (0 in front in case of len-1 hex number) 
    """

    hex_bytes = []
    for byte in int_bytes_array:
        tmp = hex(byte).replace('0x', '') # Get rid of '0x' that is generated by the cast to hex
        if len(tmp) == 1:
            tmp = f'0{tmp}' # Add 0 in front of the conversion if its len is 1
        hex_bytes.append(tmp)

    return hex_bytes
    
    
def compute_hamming_weights(sbox_out):

    """ Hamming Weights computation given the output of the SBox-lookup
    Input:
        - sbox_out: numpy array containing the result of a SBox-lookup

    Output: numpy array containing the Hamming Weights of the given SBox-lookup output  
    """

    bin_sbox_out = [np.binary_repr(out) for out in sbox_out] # Binary SBox outputs

    return [bin_out.count('1') for bin_out in bin_sbox_out] # HW = Number of 1s in bin number
    
    
    